In [105]:
import cv2
import numpy as np
from skimage.morphology import closing, disk
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.draw import polygon
from skimage.feature import canny
from scipy.ndimage.morphology import distance_transform_edt
from skimage.transform import resize
import pandas as pd
from utils.process_images import *
from utils.process_data import *

C:\Users\Taqana\AppData\Local\Temp\ipykernel_1188\3012124106.py:8: DeprecationWarning: Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
  from scipy.ndimage.morphology import distance_transform_edt


In [5]:
def process_single_image(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    thresh_img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    clean_img = closing(thresh_img > 0, disk(1))

    # Calculate stroke thickness
    thickness = stroke_thickness(clean_img)
    mean_thickness = np.mean(thickness)
    std_thickness = np.std(thickness)

    # Calculate number of pixels, end points, and intersection points
    num_pixels = np.sum(clean_img)
    num_ep = number_of_end_points(clean_img, k_nn)
    num_inters = number_of_intersection_points(clean_img, k_nn)

    # Calculate interactions between features
    interactions = {
        'mean_thickness_std_thickness': mean_thickness * std_thickness,
        'mean_thickness_num_pixels': mean_thickness * num_pixels,
        'mean_thickness_num_ep': mean_thickness * num_ep,
        'mean_thickness_num_inters': mean_thickness * num_inters,
        'std_thickness_num_pixels': std_thickness * num_pixels,
        'std_thickness_num_ep': std_thickness * num_ep,
        'std_thickness_num_inters': std_thickness * num_inters,
        'num_pixels_num_ep': num_pixels * num_ep,
        'num_pixels_num_inters': num_pixels * num_inters,
        'num_ep_num_inters': num_ep * num_inters
    }

    # Create a DataFrame with the processed features
    feature_data = {
        'mean_thickness': mean_thickness,
        'std_thickness': std_thickness,
        'num_pixels': num_pixels,
        'num_ep': num_ep,
        'num_inters': num_inters,
        **interactions  # Include interactions in the DataFrame
    }

    df = pd.DataFrame([feature_data])

    return df



In [33]:
image_path = 'drawings\\wave\\training\\healthy\\V55HO02.png'  # Replace with the path to your single image
processed_df = process_single_image(image_path)
processed_df

,mean_thickness,std_thickness,num_pixels,num_ep,num_inters,mean_thickness_std_thickness,mean_thickness_num_pixels,mean_thickness_num_ep,mean_thickness_num_inters,std_thickness_num_pixels,std_thickness_num_ep,std_thickness_num_inters,num_pixels_num_ep,num_pixels_num_inters,num_ep_num_inters
0,18.698157,13.397426,111036,4,126,250.507171,2.076169e+06,74.792629,2355.967808,1.487597e+06,53.589703,1688.075629,444144,13990536,504


In [1]:
import pickle

# Load the RF model
with open('rf_model.pickle', 'rb') as model_file:
    loaded_rf_model = pickle.load(model_file)



In [34]:
# Make predictions using the loaded model

predictions = loaded_rf_model.predict(processed_df.values )
rf_preds = loaded_rf_model.predict(processed_df.values)
rf_probs = loaded_rf_model.predict_proba(processed_df.values)
rf_probs_max = np.array([max(x, y) for x, y in rf_probs]).ravel()

# The 'predictions' variable now contains the predicted labels for each row in processed_df
print(predictions,rf_preds,rf_probs ,rf_probs_max)
#predictions,rf_preds,rf_probs ,rf_probs_max

[1] [1] [[0.21 0.79]] [0.79]


c:\Users\Taqana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Taqana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Taqana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [28]:
from PIL import Image
def get_image_size(image_path):
    try:
        img = Image.open(image_path)
        width, height = img.size
        return width, height
    except IOError as e:
        print(f"Unable to open image: {e}")
        return None, None

 # Replace with the actual path to your image
width, height = get_image_size('drawings\\wave\\training\\healthy\\V01HO02.png')

if width and height:
    print(f"Image width: {width}px")
    print(f"Image height: {height}px")

Image width: 512px
Image height: 265px


In [31]:
from PIL import Image
def get_image_size(image_path):
    try:
        img = Image.open(image_path)
        width, height = img.size
        return width, height
    except IOError as e:
        print(f"Unable to open image: {e}")
        return None, None

 # Replace with the actual path to your image
width, height = get_image_size('images\V03PO01.png')

if width and height:
    print(f"Image width: {width}px")
    print(f"Image height: {height}px")

Image width: 512px
Image height: 263px


## Last

In [110]:
import joblib
from utils.process_images import *  # Import your image processing functions
from utils.process_data import *  # Import your data processing functions

In [111]:
# Step 1: Load the saved model
model_filename = 'rf_model_wave.joblib'  # Replace with the correct filename
loaded_model = joblib.load(model_filename)


In [117]:
# Step 2: Process a new image to create the feature NumPy arraydrawingsdrawings
new_image_path = 'drawings\\wave\\training\\parkinson\\V04PO02.png'  # Replace with the path to your new image
new_image = read_and_thresh(new_image_path, resize=False)  # Process the new image as needed
# Define a function for processing a new image and making predictions
def process_new_image(new_image_path):
    new_image = read_and_thresh(new_image_path, resize=False)

    # Compute the same interaction features as during training
    mean_thickness = np.mean(stroke_thickness(closing(label_sort(new_image) > 0, disk(1))))
    std_thickness = np.std(stroke_thickness(closing(label_sort(new_image) > 0, disk(1))))
    num_pixels = sum_pixels(skeleton_drawing(new_image))
    num_ep = number_of_end_points(new_image, k_nn)
    num_inters = number_of_intersection_points(new_image, k_nn)

    # Create a feature array matching the training data
    new_features = np.array([
        mean_thickness, std_thickness, num_pixels, num_ep, num_inters,
        mean_thickness * std_thickness, mean_thickness * num_pixels, mean_thickness * num_ep, mean_thickness * num_inters,
        std_thickness * num_pixels, std_thickness * num_ep, std_thickness * num_inters,
        num_pixels * num_ep, num_pixels * num_inters, num_ep * num_inters
    ]).reshape(1, -1)

    return new_features
new_features = process_new_image(new_image_path)

In [118]:
# Step 3: Use the loaded model to make predictions
predicted_probabilities = loaded_model.predict_proba(new_features)

if predicted_probabilities[0][1] < 0.87:
    print('healthy')
else :
    print('parkinson')


parkinson


c:\Users\Taqana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
